In [1]:
#https://intellica-ai.medium.com/aspect-based-sentiment-analysis-everything-you-wanted-to-know-1be41572e238
#https://medium.datadriveninvestor.com/sentiment-analysis-is-not-enough-ffafc8bbbfeb
#https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis/notebook
#https://medium.com/analytics-vidhya/aspect-based-sentiment-analysis-a-practical-approach-8f51029bbc4a

In [1]:
!python3 -m pip install tweepy
!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_lg
!pip install stanfordnlp
!pip install textblob

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.3.0/en_core_web_sm-3.3.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
#import libraries
import pandas as pd
import string
from tqdm.notebook import tqdm_notebook
import warnings
import time
import csv
import base64
import json

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from textblob import TextBlob, Word
sns.set(color_codes=True)


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words()

import spacy
import re

# load a trained English pipeline 
nlp = spacy.load("en_core_web_sm")
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanfordnlp

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification
from transformers import pipeline


import tweepy
from tweepy import OAuthHandler

# initiate tqdm for pandas.apply() functions
tqdm_notebook.pandas()

In [ ]:
# suppress all warnings
warnings.simplefilter('ignore')

# expand notebook display options for dataframes
pd.set_option('display.max_colwidth', 200)
pd.options.display.max_columns = 999
pd.options.display.max_rows = 300

In [ ]:
# #twitter credentials
# consumer_key = "EhOZiQEfg2b8HfBeKjhX9ezB6"
# consumer_secret = "jbJcZi5g5qGSYrIGzaatDuiYPg8QpYfURdjmfbfgZZk5RUYvgb"
# access_token = "232661448-pdfmND03rvRkbPDWPfUGxuHY8ZHOENkNLSHZarPr"
# access_token_secret = "CFAcNGLqMM24Nh8m3uQglb6CSpYU5vZtTp9j4Xvd7bB5S"
# hashtag='oneplus7pro'
# tweet_count=50

# # #Authenticate credentials
# # auth = OAuthHandler(consumer_key, consumer_secret)
# # auth.set_access_token(access_token, access_secret)

# # api = tweepy.API(auth,wait_on_rate_limit=True)

# # tweet_created = []
# # tweet_text = []
# # tweets = tweepy.Cursor(api.search_tweets,q='hare krsna',count=30, lang="en")
# # for tweet in tweets.items():
# #     tweet_created.append(tweet.created_at)
# #     tweet_text.append(tweet.text)  

# # df = pd.DataFrame(columns=['created_on','tweets'])
# # df['created_on'] = tweet_created
# # df['tweets']= tweet_text

In [ ]:
# #method to fetch tweets from twitter
# def get_tweets(hashtag, consumer_key, consumer_secret, access_token, access_token_secret, tweet_count):
#     #authentication using credentials
#     auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#     auth.set_access_token(access_token, access_token_secret)
#     api = tweepy.API(auth,wait_on_rate_limit=True)

#     #fetching tweets holding specific hashtag
#     tweet_text, tweet_created = list(), list()
#     for tweet in tweepy.Cursor(api.search_tweets,q=hashtag, count=tweet_count, lang="en").items():
#         tweet_text.append(tweet.text.encode('utf-8'))
#         tweet_created.append(tweet.created_at)
       
#     #creating dataframe    
#     df = pd.DataFrame({'tweets': tweet_text, 'created_on': tweet_created})
#     return df

In [ ]:
# twitter_content = get_tweets(hashtag, consumer_key, consumer_secret, access_token, access_token_secret, tweet_count=50)

In [ ]:
df = pd.read_csv('databricks2-Copy1.csv',usecols=['title'])
df.head()

# Data Cleaning
Extracted tweet text may contain hashtags, tagging account, emojis, web-site/HTTP links, and HTML tags that need to be removed in pre-processing steps.

In [ ]:
!pip install urlextract

In [ ]:
import re
from urlextract import URLExtract
extractor = URLExtract()
import contractions
import nltk
nltk.download('brown')
# dictionary = {'" "':  ' ', ",": ' ','-' : ' ',' " ': ' ','()' : '',
#                   '%' : '','#': '','\[':'','\]': '',
#                   '\\(\\)':'','\*':' ' ,'\|':'',
#                  '!':'','\?':'','\(\)': '','/': '',
#                   '\)' : '', '\(' : '', '"': '', ':':'',"'": '','  +':' ',
#                  }

def preprocess(text):
    
    #text = text.replace(dictionary,regex=True)
    #to lowercase
    text = str(text).lower()
    #remove urls
    urls = extractor.find_urls(text)
    for url in urls:
        text = text.replace(url, '')
    #remove new lines
    text = text.replace('\\n',' ')
    #replacement
    text = text.replace(re.escape("\]\["), "")
    text = text.replace(re.escape("]"), "")
    #remove none meaningful    
    text = re.sub(r'\\x[0-9a-f]{2}', '',text)
    #remove emails
    text = re.sub(r'\S*@\S*\s?',' ',text)
    #remove mentions
    text = re.sub(r'@\S+', ' ', text)
    #contractions
    text = contractions.fix(text)
    #remove hashtags
    text = re.sub(r'@\S+', ' ', text)
    #remove emojis
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    #remove retweets
    text = text.replace(r'rt', '')
    #remove all punct
    text = re.sub('[^A-za-z0-9]', ' ', text)
    #remove extras whitespaces
    text = re.sub('\s+', ' ', text)
    #text = cleaning(text)
    return text

In [ ]:
# df['title'] = preprocess(df['title'])

In [ ]:
df['title'] = df['title'].apply(preprocess)

In [ ]:
def cleaning (column):
    dictionary = {'" "':  ' ', ",": ' ','-' : ' ',' " ': ' ','()' : '',
                  '%' : '','#': '','\[':'','\]': '',
                  '\\(\\)':'','\*':' ' ,'\|':'',
                 '!':'','\?':'','\(\)': '','/': '',
                  '\)' : '', '\(' : '', '"': '', ':':'',"'": '','  +':' ',
                 }
    column = column.replace(dictionary, regex=True)
    #column = column.str.lower()
    
    return column

In [ ]:
df['title'] = cleaning(df['title'])

In [ ]:
result=df['title'].to_list()

In [ ]:
result

In [ ]:
# Check sentiment polarity of each sentence.
sentiment_scores = list()
i = 0
for sentence in result:
    line = TextBlob(sentence)
    sentiment_scores.append(line.sentiment.polarity)
    if(i <= 10):
        print(sentence + ": POLARITY=" + str(line.sentiment.polarity))
        i += 1

In [ ]:
# Let's plot these sentiments!
sns.distplot(sentiment_scores)

In [ ]:
comments = TextBlob(' '.join(result))

In [ ]:
# Check out noun phrases, will be useful for frequent feature extraction
comments.noun_phrases

In [ ]:
#compactness pruning:
cleaned = list()
for phrase in comments.noun_phrases:
    count = 0
    for word in phrase.split():
        # Count the number of small words and words without an English definition
        if len(word) <= 2 or (not Word(word).definitions):
            count += 1
    # Only if the 'nonsensical' or short words DO NOT make up more than 40% (arbitrary) of the phrase add
    # it to the cleaned list, effectively pruning the ones not added.
    if count < len(phrase.split())*0.4:
        cleaned.append(phrase)
        
print("After compactness pruning:\nFeature Size:")
len(cleaned)

In [ ]:
for phrase in cleaned:    
    match = list()
    temp = list()
    word_match = list()
    for word in phrase.split():
        # Find common words among all phrases
        word_match = [p for p in cleaned if re.search(word, p) and p not in word_match]
        # If the size of matched phrases set is smaller than 30% of the cleaned phrases, 
        # then consider the phrase as non-redundant.
        if len(word_match) <= len(cleaned)*0.3 :
            temp.append(word)
            match += word_match
            
    phrase = ' '.join(temp)
    print("Match for " + phrase + ": " + str(match))

    if len(match) >= len(cleaned)*0.1 :
        # Redundant feature set, since it contains more than 10% of the number of phrases. 
        # Prune all matched features.
        for feature in match:
            if feature in cleaned:
                cleaned.remove(feature)
            
        # Add largest length phrase as feature
        cleaned.append(max(match, key=len))
        

print("After redundancy pruning:\nFeature Size:" + str(len(cleaned)))
print("Cleaned features:")
cleaned

In [ ]:
from nltk.corpus import stopwords
feature_count = dict()
for phrase in cleaned:
    count = 0
    for word in phrase.split():
        if word not in stopwords.words('english'):
            count += comments.words.count(word)
    
    print(phrase + ": " + str(count))
    feature_count[phrase] = count

In [ ]:
# Select frequent feature threshold as (max_count)/100 
# This is an arbitrary decision as of now.
counts = list(feature_count.values())
features = list(feature_count.keys())
threshold = len(comments.noun_phrases)/100

print("Threshold:" + str(threshold))

frequent_features = list()

for feature, count in feature_count.items():
    if count >= threshold:
        frequent_features.append(feature)
        
print('Frequent Features:')
frequent_features

In [ ]:
feats = [feat for feat in features][:20]
cnt = [count for count in counts][:20]

In [ ]:
# Let's plot these feature occurences and draw the threshold line
sns.set()
sns.set_context("poster")
f, ax = plt.subplots(figsize=(10, 13))
sns.swarmplot(y=feats, x=cnt, color="c", ax=ax)
plt.plot([threshold, threshold], [0, len(features)], linewidth=4, color="r")

In [ ]:
#https://github.com/mlp9/Aspect-based-opinion-mining-NLP-with-Python-/blob/main/aspect_based_opinion_mining.ipynb

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score
#b=dataset.values.T.tolist()
#print(b)
nltk_results = [nltk_sentiment(row) for row in frequent_features]
#print(nltk_results)
results_df = pd.DataFrame(nltk_results)
#print(results_df)
text_df = pd.DataFrame(frequent_features)
#print(text_df)
nltk_df = text_df.join(results_df)
#nltk_df1=nltk_df[[0,'neu']]
#print(nltk_df)

In [ ]:
nltk_df.head(5)

In [ ]:
newdf=nltk_df[0]
newdf=pd.DataFrame({'features':nltk_df[0],'pos':nltk_df['pos'],'neg':nltk_df['neg']})
newdf.pos=newdf.pos+0.2
newdf.neg=newdf.neg-0.2
newdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
newdf

#noun=['positive attitude','good job knowledge','team player','customer acquisition','good visibility','back office process','successful launch','soft skills','suitable candidates','core issues']
#pos_l=[0.71,0.42,0.3,0.2,0.2,0.6,0.2,0.5,0.43,0]
#neg_l=[-0.1,0,-0.300,0,-0.1,0,-0.1,-0.2,0,-0.3]

pos=newdf[0:5]['pos']
neg=newdf[0:5]['neg']

# data to plot
n_groups = 5
positive =newdf['pos'].head(5) 
negative =newdf['neg'].head(5)
 
# create plot
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.3
opacity = 1
 
rects1 = plt.bar(index, positive, bar_width,
alpha=opacity,
color='b',
label='positive sentiments')
 
rects2 = plt.bar(index + bar_width, negative, bar_width,
alpha=opacity,
color='r',
label='negative sentiments')
 
plt.xlabel('Features')
plt.ylabel('sentiment value')
plt.title('Top features and its sentiment')
plt.xticks(index + bar_width, newdf['features'].head(5))
plt.xticks(rotation = 90)
plt.legend()
fig.set_size_inches(20, 15)
plt.show()

In [ ]:
absa_list = dict()
# For each frequent feature
for f in frequent_features:
    # For each comment
    absa_list[f] = list()
    for comment in result:
        blob = TextBlob(comment)
        # For each sentence of the comment
        for sentence in blob.sentences:
            # Search for frequent feature 'f'
            q = '|'.join(f.split())
            if re.search(r'\w*(' + str(q) + ')\w*', str(sentence)):
                absa_list[f].append(sentence)

In [ ]:
absa_list

In [ ]:
scores = list()
absa_scores = dict()
for k, v in absa_list.items():
    absa_scores[k] = list()
    for sent in v:
        score = sent.sentiment.polarity
        scores.append(score)
        absa_scores[k].append(score)

In [ ]:
# Now that we have all the scores, let's plot them!
# For comparison, we replot the previous global sentiment polarity plot
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(20, 10))
plot1 = sns.distplot(scores, ax=ax1)

ax1.set_title('Aspect wise scores')
ax1.set_xlabel('Sentiment Polarity')
ax1.set_ylabel('# of comments')

ax2.set_title('Comment wise scores')
ax2.set_xlabel('Sentiment Polarity')
ax2.set_ylabel('# of comments')

plot2 = sns.distplot(sentiment_scores, ax=ax2)

In [ ]:
# Create data values for stripplot and boxplot
vals = dict()
vals["aspects"] = list()
vals["scores"] = list()
for k, v in absa_scores.items():
    for score in v:
        vals["aspects"].append(k)
        vals["scores"].append(score)

In [ ]:
vals2 = {
    "aspects" : vals["aspects"][:1500],
    "scores" : vals["scores"][:1500]
}



In [ ]:
vals2


In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 10))

color = sns.color_palette("Blues", 6)
plt.xticks(rotation=90)
sns.set_context("paper", font_scale=3) 
sns.boxplot(x="aspects", y="scores", data=vals2, palette=color, ax=ax1)

In [ ]:
color = sns.color_palette("Reds", 6)
fig, ax1 = plt.subplots(figsize=(30, 10))
plt.xticks(rotation=90)
sns.set_context("paper", font_scale=2) 
sns.stripplot(x="aspects", y="scores",data=vals2, palette=color)